In [ ]:
with open('/content/drive/MyDrive/egypt.txt','r') as f:
     summary = f.read()

In [ ]:
pip install git+https://github.com/boudinfl/pke.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-__qycyv2
  Running command git clone -q https://github.com/boudinfl/pke.git /tmp/pip-req-build-__qycyv2
     |████████████████████████████████| 235 kB 14.2 MB/s 
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160276 sha256=440d51e1be8ca3c65c8ae4e6a11af4b2621fd0b2a1cfa5de15cd15564c876fb8
  Stored in directory: /tmp/pip-ephem-wheel-cache-gie11avk/wheels/fa/b3/09/612ee93bf3ee4164bcd5783e742942cdfc892a86039d3e0a33
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=dbac766ed49bce9b2171a524acc18d77feae53cf936253df29ef216a4e3d8220
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built pke sklearn


In [ ]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import string
import pke
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')


def extracting_keywords(summary):
    keywords = []
    extractor = pke.unsupervised.MultipartiteRank()
    stoplist = list(string.punctuation)
    stoplist+= stopwords.words('english')
    stoplist+= ['-lrb-', '-rrb-', '-lcb-', '-rcb-', '-lsb-', '-rsb-']
    extractor.load_document(summary,stoplist=stoplist)
    pos = {'PROPN'}
    extractor.candidate_selection(pos=pos)
   
    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best(n=15)
    for i in keyphrases:
        keywords.append(i[0])
    return keywords

key=extracting_keywords(summary)
print(key)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value ['ner', 'textcat', 'parser'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


['egyptians', 'nile river', 'egypt', 'nile', 'euphrates', 'tigris', 'old kingdom', 'red land', 'double crown', 'narmer', 'upper', 'longest river', 'crown', 'africa', 'mediterranean sea']


In [ ]:
pip install flashtext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

def tokenize_sentences(text):
    sentences = [sent_tokenize(text)]
    sentences = [y for x in sentences for y in x]
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_sentences_for_keyword(key, sentences):
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in key:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

sentences = tokenize_sentences(summary)
keyword_sentence_mapping = get_sentences_for_keyword(key, sentences)
        
print (keyword_sentence_mapping)

{'egyptians': ['Egyptians cut the stems into strips, pressed them, and dried them into sheets that could be rolled into scrolls.', 'These beautiful blue stones were used in jewelry.The Nile had fish and other wildlife that Egyptians wanted.', 'The Nile provided so well for Egyptians that sometimes they had surpluses, or more goods than they needed.', 'For example, some ancient Egyptians learned to be scribes, people whose job was to write and keep records.', 'Egyptian Houses Egyptians built houses using bricks made of mud from the Nile mixed with chopped straw.', 'Egyptians believed that if a tomb was robbed, the person buried there could not have a happy afterlife.', 'The Egyptians also developed a paperlike material called papyrus papyrus from a reed of the same name.', 'Egyptians believed that if the pharaoh and his subjects honored the gods, their lives would be happy.', 'Their army conquered by using better weapons and horse-drawn chariots, which were new to Egyptians.', 'Red Land

In [ ]:
pip install pywsd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import requests
import json
import re
import random
from pywsd.similarity import max_similarity
from pywsd.lesk import adapted_lesk
from pywsd.lesk import simple_lesk
from pywsd.lesk import cosine_lesk

from nltk.corpus import wordnet as wn

# Distractors from Wordnet
def get_distractors_wordnet(syn,word):
    distractors=[]
    word= word.lower()
    orig_word = word
    if len(word.split())>0:
        word = word.replace(" ","_")
    hypernym = syn.hypernyms()
    if len(hypernym) == 0: 
        return distractors
    for item in hypernym[0].hyponyms():
        name = item.lemmas()[0].name()
        #print ("name ",name, " word",orig_word)
        if name == orig_word:
            continue
        name = name.replace("_"," ")
        name = " ".join(w.capitalize() for w in name.split())
        if name is not None and name not in distractors:
            distractors.append(name)
    return distractors

def get_wordsense(sent,word):
    word= word.lower()
    
    if len(word.split())>0:
        word = word.replace(" ","_")
    
    
    synsets = wn.synsets(word,'n')
    if synsets:
        wup = max_similarity(sent, word, 'wup', pos='n')
        adapted_lesk_output =  adapted_lesk(sent, word, pos='n')
        lowest_index = min (synsets.index(wup),synsets.index(adapted_lesk_output))
        return synsets[lowest_index]
    else:
        return None

# Distractors from http://conceptnet.io/
def get_distractors_conceptnet(word):
    word = word.lower()
    original_word= word
    if (len(word.split())>0):
        word = word.replace(" ","_")
    distractor_list = [] 
    url = "http://api.conceptnet.io/query?node=/c/en/%s/n&rel=/r/PartOf&start=/c/en/%s&limit=5"%(word,word)
    obj = requests.get(url).json()

    for edge in obj['edges']:
        link = edge['end']['term'] 

        url2 = "http://api.conceptnet.io/query?node=%s&rel=/r/PartOf&end=%s&limit=10"%(link,link)
        obj2 = requests.get(url2).json()
        for edge in obj2['edges']:
            word2 = edge['start']['label']
            if word2 not in distractor_list and original_word.lower() not in word2.lower():
                distractor_list.append(word2)
                   
    return distractor_list

key_distractor_list = {}

for keyword in keyword_sentence_mapping:
    wordsense = get_wordsense(keyword_sentence_mapping[keyword][0],keyword)
    if wordsense:
        distractors = get_distractors_wordnet(wordsense,keyword)
        if len(distractors) ==0:
            distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors
    else:
        
        distractors = get_distractors_conceptnet(keyword)
        if len(distractors) != 0:
            key_distractor_list[keyword] = distractors

index = 1
for each in key_distractor_list:
    sentence = keyword_sentence_mapping[each][0]
    pattern = re.compile(each, re.IGNORECASE)
    output = pattern.sub( " _______ ", sentence)
    print ("%s)"%(index),output)
    choices = [each.capitalize()] + key_distractor_list[each]
    top4choices = choices[:4]
    random.shuffle(top4choices)
    optionchoices = ['a','b','c','d']
    for idx,choice in enumerate(top4choices):
        print ("\t",optionchoices[idx],")"," ",choice)
    print ("\nMore options: ", choices[4:20],"\n\n")
    index = index + 1


1)  _______  cut the stems into strips, pressed them, and dried them into sheets that could be rolled into scrolls.
	 a )   Bantu
	 b )   Angolan
	 c )   Algerian
	 d )   Egyptians

More options:  ['Basotho', 'Beninese', 'Berber', 'Black African', 'Burundian', 'Cameroonian', 'Carthaginian', 'Chadian', 'Chewa', 'Congolese', 'Djiboutian', 'Egyptian', 'Ethiopian', 'Eurafrican', 'Ewe', 'Fulani'] 


2) As in many ancient societies, much of the knowledge of  _______  came about as priests studied the world to find ways to please the gods.
	 a )   Saudi Arabia
	 b )   Egypt
	 c )   Iraq
	 d )   Kuwait

More options:  ['Jordan', 'Israel', 'Fertile Crescent', 'Turkey', 'Iran', 'Lebanon', 'Shari', 'Mauritania', 'Nigeria', 'Somali peninsula', 'Sierra Leone', 'Malawi', 'North Africa', 'Senegal', 'Mozambique', 'Lake Tanganyika'] 


3) About 5,000 years ago, they noticed that a star now called Sirius appeared shortly before the  _______  began to flood.
	 a )   Buganda
	 b )   Entebbe
	 c )   Gulu
	